<a href="https://colab.research.google.com/github/fariasfranco/ClasificadordeCanciones/blob/main/Clasificador_de_canciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Paso 1: Inicio Programa
####En este primer paso se importan las bibliotecas necesarias para el proyecto, Estas bibliotecas incluyen "pandas" para el manejo de datos, "train_test_split" para dividir los datos en conjuntos de entrenamiento y prueba, y varias bibliotecas de modelos de aprendizaje automático como "KNeighborsClassifier", "SVC", "DecisionTreeClassifier", "GaussianNB", y "VotingClassifier".Donde tambien se utiliza "GridSearchCV" para realizar una búsqueda de cuadrícula en el ajuste de hiperparámetros y varias métricas de evaluación de modelos, como "accuracy_score", "confusion_matrix", "precision_score", "recall_score" y "f1_score".
####Luego se carga el archivo de base de datos .CSV  llamado "Canciones_Spotify.csv" en un DataFrame de "pandas" llamado "data0".

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import numpy as np  # Importa la biblioteca numpy

#Cargar el conjunto de datos
csv_file_url = "/content/Canciones_Spotify.csv"
data0 = pd.read_csv(csv_file_url)


##Paso 2: PreProcesamiento de Datos
####En este paso, se ha preparado el conjunto de datos para que pueda ser utilizado en la construcción de modelos de machine learning. Las características se han codificado y separado de las etiquetas, lo que es esencial para el entrenamiento y evaluación de los modelos.Lo cual esto implica la codificación de "variables categóricas" y la "separación de las características" (variables independientes) y las etiquetas (variable dependiente).
####"Variables Categóricas": En este paso, se toman las columnas "artist" y "song_title", que son variables categóricas, y se aplican técnicas de codificación. En este caso, se utiliza el método conocido como "one-hot encoding". Lo que hace esta técnica es convertir cada categoría única en estas columnas en una nueva columna binaria. Cada columna binaria representa si una canción pertenece a una categoría particular o no. Esto ayuda a los modelos de machine learning a trabajar con estas características categóricas.
####"Separación de Características (X) y Etiquetas (y)": Después de codificar las variables categóricas, el conjunto de datos se divide en dos partes:

####X: Este es el conjunto de características. Contiene todas las columnas excepto la columna "target", que es la etiqueta que queremos predecir. En otras palabras, X contiene las características que se utilizarán para hacer predicciones sobre las preferencias de las canciones.
####y: Esta es la etiqueta que se quiere predecir. En este caso, la columna "target" que indica si una canción fue del agrado del usuario (1) o no (0).

In [ ]:
# Preprocesamiento de datos
# Codificar variables categóricas (artist y song_title) utilizando one-hot encoding
data = pd.get_dummies(data0, columns=["artist", "song_title"])

# Separar características (X) y etiquetas (y)
X = data.drop(columns=["target"])
y = data["target"]


##Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
####En ese terceer paso dividimos los datos en conjuntos de entrenamiento y prueba. Utilizamos la función "train_test_split" de la biblioteca "scikit-learn" para realizar esta división.El conjunto de entrenamiento (X_train y y_train) se utiliza para entrenar nuestros modelos de machine learning, mientras que el conjunto de prueba (X_test y y_test) se utiliza para evaluar el rendimiento de los modelos.
####1.X_train:contiene las características del conjunto de entrenamiento.
####2.X_test:contiene las características del conjunto de prueba.
####3.y_train:contiene las etiquetas del conjunto de entrenamiento.
####4.y_test:contiene las etiquetas del conjunto de prueba.
####Se configuro el parametro "test_size" en 0.2, lo que significa que el 20% de los datos se utilizarán como conjunto de prueba, y el 80% restante se utilizará como conjunto de entrenamiento. También se establecio "random_state" en 42 para garantizar la reproducibilidad de la división de datos.

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##Paso 4: Entrenar modelos de machine learning
####Se entrena algunos modelos de machine learning.Primero,se definio una variable llamada "evaluate_model" que ayuda a evaluar la precisión de un modelo utilizando validación cruzada (cross_val_score) con 5 particiones. Luego, se importa varios modelos de "scikit-learn" que incluyen "K-Nearest Neighbors (KNN)", "Support Vector Machines (SVM)", "Árbol de Decisión", "Naive Bayes" y "Random Forest".
####En este paso, también se selecciona un conjunto de características importantes para entrenar nuestros modelos. Estas características incluyen "danceability", "energy", "valence", "acousticness", "instrumentalness", "tempo" y "speechiness".
####Después de filtrar los conjunto de datos de entrenamiento y prueba con estas características seleccionadas, se itera a través de cada modelo y se calcula su precisión utilizando la función "evaluate_model". Luego, hemos realizado una búsqueda de hiperparámetros (Grid Search) para el modelo KNN para encontrar los mejores valores para los hiperparámetros 'n_neighbors', 'weights' y 'p'. Los mejores modelos se han almacenado en el diccionario "best_models".

In [ ]:
from sklearn.model_selection import cross_val_score
def evaluate_model(model, X, y):
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
svm = SVC()
decision_tree = DecisionTreeClassifier()
naive_bayes = GaussianNB()
random_forest = RandomForestClassifier()

# Define las características que deseas utilizar
important_features = ["danceability", "energy", "valence", "acousticness", "instrumentalness", "tempo", "speechiness"]

# Filtra el conjunto de datos con las características seleccionadas
X_train = X_train[important_features]
X_test = X_test[important_features]

models = {
    "KNN": knn,
    "SVM": svm,
    "Decision Tree": decision_tree,
    "Naive Bayes": naive_bayes,
    "Random Forest": random_forest,
}

best_models = {}  # Almacenar los mejores modelos para cada tipo de modelo

for model_name, model in models.items():
    accuracy = evaluate_model(model, X_train, y_train)
    print(f"{model_name} Accuracy: {accuracy}")

param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)
best_models[model_name] = grid_search.best_estimator_
best_models

KNN Accuracy: 0.5815145280079995
SVM Accuracy: 0.5337884352825799
Decision Tree Accuracy: 0.6763744399361575
Naive Bayes Accuracy: 0.6317308616810569
Random Forest Accuracy: 0.7526508086071957


{'Random Forest': KNeighborsClassifier(n_neighbors=3, p=1, weights='distance')}

##Paso 5: Ajustar y evaluar modelos de machine learning
####Este Paso 5 se enfoca en ajustar y evaluar los modelos de machine learning que has entrenado anteriormente.
####evaluate_model_performance: Esta función evalúa el rendimiento de un modelo. Calcula la matriz de confusión, la precisión, la recuperación y el puntaje F1 del modelo en el conjunto de prueba.

####Loop sobre los modelos: Un bucle for se utiliza para iterar a través de los modelos que has entrenado en el Paso 4. Para cada modelo:

#####Se ajusta el modelo utilizando los datos de entrenamiento con model.fit(X_train, y_train).

#####Se evalúa el rendimiento del modelo en el conjunto de prueba utilizando evaluate_model_performance.

#####Los resultados, incluyendo la matriz de confusión, precisión, recuperación y puntaje F1, se imprimen para cada modelo.

####Seguimiento del mejor modelo: Se realiza un seguimiento del mejor modelo en términos de puntaje F1. Si un modelo tiene un puntaje F1 más alto que el actual mejor modelo, se actualiza best_model_name con el nombre del nuevo mejor modelo y se almacena el modelo en el diccionario best_models.

####Mejor modelo: Finalmente, se imprime el nombre del mejor modelo basado en el puntaje F1.
####En resumen este paso ayuda a identificar cuál de los modelos tiene el mejor rendimiento en términos de F1-score, lo que puede ser útil para seleccionar el modelo final.

In [ ]:
def evaluate_model_performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return conf_matrix, precision, recall, f1

best_models = {}  # Almacenar los mejores modelos para cada tipo de modelo
best_f1 = 0  # Almacenar el mejor valor de F1
best_model_name = None  # Almacenar el nombre del mejor modelo

for model_name, model in models.items():
    # Ajustar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    # Evaluar el rendimiento del modelo en el conjunto de prueba
    conf_matrix, precision, recall, f1 = evaluate_model_performance(model, X_test, y_test)
    print(f"{model_name} Performance:")
    print("Confusion Matrix:\n", conf_matrix)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

    # Si es el mejor modelo hasta el momento, actualiza best_model_name y best_f1
    if best_model_name is None or f1 > best_f1:
        best_f1 = f1
        best_model_name = model_name

    # Almacena el mejor modelo en el diccionario best_models
    best_models[model_name] = model

print(f"Mejor modelo: {best_model_name}")

KNN Performance:
Confusion Matrix:
 [[130  76]
 [ 84 114]]
Precision: 0.6
Recall: 0.5757575757575758
F1-score: 0.5876288659793815
SVM Performance:
Confusion Matrix:
 [[ 53 153]
 [ 41 157]]
Precision: 0.5064516129032258
Recall: 0.7929292929292929
F1-score: 0.6181102362204725
Decision Tree Performance:
Confusion Matrix:
 [[139  67]
 [ 59 139]]
Precision: 0.6747572815533981
Recall: 0.702020202020202
F1-score: 0.6881188118811882
Naive Bayes Performance:
Confusion Matrix:
 [[121  85]
 [ 60 138]]
Precision: 0.6188340807174888
Recall: 0.696969696969697
F1-score: 0.655581947743468
Random Forest Performance:
Confusion Matrix:
 [[151  55]
 [ 50 148]]
Precision: 0.729064039408867
Recall: 0.7474747474747475
F1-score: 0.7381546134663343
Mejor modelo: Random Forest


##Paso 6: Realizar un ensamble de los modelos (Votación Mayoritaria)
####Este paso crea un modelo de ensamble que combina las predicciones de varios modelos individuales. El ensamble puede ayudar a mejorar el rendimiento y la estabilidad de las predicciones finales.
####Votación Mayoritaria: La votación mayoritaria es un método de ensamble que combina las predicciones de varios modelos de machine learning para tomar una decisión final. En este caso, se utiliza la clase "VotingClassifier" de "scikit-learn" para implementar esta técnica.

####Configuración del Ensamble: se configura el ensamble con varios modelos, incluyendo KNN, SVM, Árbol de Decisión, Naive Bayes y Random Forest. Los modelos se pasan como estimadores en una lista. La opción voting="hard" indica que se tomará la decisión final basada en la mayoría de votos de los modelos.

####Ajuste del Ensamble: se ajusta el modelo de ensamble con los datos de entrenamiento utilizando ensemble.fit(X_train, y_train).

In [ ]:
ensemble = VotingClassifier(estimators=[("KNN", best_models["KNN"]), ("SVM", best_models["SVM"]), ("Decision Tree", best_models["Decision Tree"]), ("Naive Bayes", best_models["Naive Bayes"]), ("Random Forest", best_models["Random Forest"])], voting="hard")
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('KNN', KNeighborsClassifier()), ('SVM', SVC()),
                             ('Decision Tree', DecisionTreeClassifier()),
                             ('Naive Bayes', GaussianNB()),
                             ('Random Forest', RandomForestClassifier())])

##Paso 7: Evaluación y análisis del rendimiento del ensamble
####Este paso proporciona una evaluación completa del rendimiento del ensamble, lo que te permite comprender cuán efectivo es en la tarea de categorizar canciones según las preferencias del usuario.

In [ ]:
conf_matrix, precision, recall, f1 = evaluate_model_performance(ensemble, X_test, y_test)
print("Ensemble Performance:")
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Ensemble Performance:
Confusion Matrix:
 [[133  73]
 [ 43 155]]
Precision: 0.6798245614035088
Recall: 0.7828282828282829
F1-score: 0.727699530516432


##Extras
####Este paso proporciona una experiencia práctica al usuario, ya que simula la funcionalidad de una aplicación de recomendación de canciones.
####Selección de una Canción Aleatoria: El código elige aleatoriamente una canción de las que le gustan al usuario a partir del conjunto de datos. Esto simula la idea de seleccionar una canción de la que al usuario le guste para luego recibir recomendaciones similares.

####Predicción de Canciones Sugeridas: Utiliza el mejor modelo de aprendizaje automático entrenado anteriormente para predecir una canción sugerida basada en las características de la canción aleatoria elegida. El modelo toma las características de la canción aleatoria y hace una predicción sobre qué otras canciones le podrían gustar al usuario.

####Selección de Canciones Sugeridas Aleatorias: El código crea una lista de canciones sugeridas que se predice que le gustarán al usuario. Luego, selecciona una de estas canciones sugeridas al azar para recomendar.

####Impresión de Resultados: Imprime la canción que le gusta al usuario y la canción sugerida, si se encuentra una recomendación adecuada. Si no hay recomendaciones disponibles, muestra un mensaje informativo

In [ ]:
import random
import numpy as np

# Elegir una canción aleatoria de las que le gusta al usuario desde el conjunto de datos
canciones_gustadas = data0[data0["target"] == 1][["song_title", "artist"]]
canciones_gustadas = list(canciones_gustadas.itertuples(index=False, name=None))
cancion_aleatoria_info = random.choice(canciones_gustadas)
cancion_aleatoria = cancion_aleatoria_info[0]
artista_cancion_gusta = cancion_aleatoria_info[1]

# Imprimir la canción elegida que le gusta al usuario
print(f"Canción que te gusta: {cancion_aleatoria} (Artista: {artista_cancion_gusta})")

# Obtener las características de la canción aleatoria
cancion_aleatoria_features = data0[data0["song_title"] == cancion_aleatoria][important_features]

# Utilizar el mejor modelo de aprendizaje automático para predecir una canción sugerida
resultado_predicción = best_models[best_model_name].predict(cancion_aleatoria_features)

# Obtener una lista de canciones que el modelo predice que le gustarán al usuario
canciones_sugeridas = []

for song_title, artist in zip(data0["song_title"], data0["artist"]):
    song_features = data0[data0["song_title"] == song_title][important_features]
    prediction = best_models[best_model_name].predict(song_features)
    if np.any(prediction == 1) and song_title != cancion_aleatoria:
        canciones_sugeridas.append((song_title, artist))

# Seleccionar una de las canciones sugeridas aleatoriamente
if canciones_sugeridas:
    cancion_sugerida = random.choice(canciones_sugeridas)
    print(f"Canción sugerida: {cancion_sugerida[0]} (Artista: {cancion_sugerida[1]})")
else:
    print("Lo sentimos, no tenemos recomendaciones en este momento.")


Canción que te gusta: Don't Mean A Thing (Artista: Lapalux)
Canción sugerida: Myth (Artista: Beach House)
